<a href="https://colab.research.google.com/github/kwojtylko/flat_price_estimator2022/blob/main/1_web_scraping_%26_data_preparation2022.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Predykcja ceny 1 m2 mieszkania  na rynku pierwotym w Krakowie w maju 2022 r. na podstawie ofert zamieszczonych na portalu gratka.pl 

In [26]:
# import bibliotek
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd
import numpy as np

# Pobranie danych

In [27]:
def get_offers_gratka(real_estate="mieszkania", city="krakow", market="pierwotny"):

  # tworzenie listy z nazwami cech
  features  = [ ["url_base", 
                "url_page", 
                "page",
                "location",  
                "title", 
                "link",
                "description",
                "area_m2", 
                "n_rooms",
                "n_floor", 
                "price_per_m2" 
             ] ]


  # pobranie liczby dostępnych stron w wyniku wyszukiwania
  
  URL_base = "https://gratka.pl/nieruchomosci/" + real_estate + "/" + city + "/" + market
  req_url_base = requests.get(URL_base)
  html_url_base = req_url_base.content
  soup_url_base = BeautifulSoup(html_url_base, "lxml")
  n_pages = 168
  


  # pętla do pobrania wszystkich stron
  for page in range(1, n_pages+1):
    time.sleep(1)

    # definiowanie żądania
    URL_page = URL_base + "?page=" + str(page)
    req_page = requests.get(URL_page) 
    print("Strona:", page, "|", "Status:", req_page, sep=" ", end="\n")

    # pobranie strony html
    html_page = req_page.content

    # tworzenie obiektu BeautifulSoup
    soup_page = BeautifulSoup(html_page, "lxml")

    # lista z informacjami o ofertach ze sparsowanej strony
    offers = soup_page.find_all("article", {"class": "teaserUnified"})
    print("Lista ofert na stronie:", len(offers), "| Strona: [", page, "/", n_pages, "]", sep=" ", end="\n")

    for offer in offers:

      # pobranie lokalizacji oferty
      location = offer.find("span", {"class": "teaserUnified__location"}).text.strip()

      # pobranie tytułu oferty
      title = offer.find("a", {"class": "teaserUnified__anchor"}).text.strip()

      # pobranie linku do oferty
      link = offer.find("a").get("href")

      # pobranie opisu oferty
      description = np.NaN if offer.find("p", {"class": "teaserUnified__description"})==None else offer.find("p", {"class": "teaserUnified__description"}).text.strip()

      # pobranie ceny zł/m2 z oferty
      price_per_m2 = np.NaN if offer.find("span", {"class": "teaserUnified__additionalPrice"})==None else offer.find("span", {"class": "teaserUnified__additionalPrice"}).text.strip("zł/m²").strip("\n").replace(" ","").replace(",",".") 

      # pobranie informacji o metrazu, liczbie pokoi oraz nr piętra
      offer_details = offer.find_all("li", {"class": "teaserUnified__listItem"})
      
      # tworzenie pętli dla uzupełnienia listy z dodatkową informacją
      # o metrazu, liczbie pokoi oraz nr piętra
      offer_detail_list = []
      for offer_detail in offer_details:
          offer_detail_list.append(offer_detail.text)
          #print(offer_detail_list)
      
      # pobranie metraza
      area_m2 = "".join(list(filter(lambda i: "m2" in i, offer_detail_list))).replace(" m2", "").replace(",",".")

      # pobranie liczby pokoi
      n_rooms = "".join(list(filter(lambda i: " pok" in i, offer_detail_list))).replace(" pokoje", "").replace(" pokój", "").replace(" pokoi", "")
      
      # pobranie nr piętra       
      n_floor = 0 if "parter" in offer_detail_list else "".join(list(filter(lambda i: "piętro" in i, offer_detail_list))).replace(" piętro", "")



      features.append([URL_base, 
                URL_page, 
                page,
                location,  
                title, 
                link,
                description,
                area_m2, 
                n_rooms,
                n_floor, 
                price_per_m2 
             ])
      
  # przekształcenie zbioru danych w DF 
  return(pd.DataFrame(features[1:], columns=features[0]))

In [28]:
# przygotowanie df
gratka_df = get_offers_gratka()

Strona: 1 | Status: <Response [200]>
Lista ofert na stronie: 32 | Strona: [ 1 / 168 ]
Strona: 2 | Status: <Response [200]>
Lista ofert na stronie: 32 | Strona: [ 2 / 168 ]
Strona: 3 | Status: <Response [200]>
Lista ofert na stronie: 32 | Strona: [ 3 / 168 ]
Strona: 4 | Status: <Response [200]>
Lista ofert na stronie: 32 | Strona: [ 4 / 168 ]
Strona: 5 | Status: <Response [200]>
Lista ofert na stronie: 32 | Strona: [ 5 / 168 ]
Strona: 6 | Status: <Response [200]>
Lista ofert na stronie: 32 | Strona: [ 6 / 168 ]
Strona: 7 | Status: <Response [200]>
Lista ofert na stronie: 32 | Strona: [ 7 / 168 ]
Strona: 8 | Status: <Response [200]>
Lista ofert na stronie: 32 | Strona: [ 8 / 168 ]
Strona: 9 | Status: <Response [200]>
Lista ofert na stronie: 32 | Strona: [ 9 / 168 ]
Strona: 10 | Status: <Response [200]>
Lista ofert na stronie: 32 | Strona: [ 10 / 168 ]
Strona: 11 | Status: <Response [200]>
Lista ofert na stronie: 32 | Strona: [ 11 / 168 ]
Strona: 12 | Status: <Response [200]>
Lista ofert 

In [29]:
# przegląd danych
gratka_df

,url_base,url_page,page,location,title,link,description,area_m2,n_rooms,n_floor,price_per_m2
0,https://gratka.pl/nieruchomosci/mieszkania/krakow/pierwotny,https://gratka.pl/nieruchomosci/mieszkania/krakow/pierwotny?page=1,1,"Kraków, małopolskie","Nowe mieszkanie Kraków, ul. Borowego",https://gratka.pl/nieruchomosci/nowe-mieszkanie-krakow-ul-borowego/ob/23636851,NaN,97.70,5,1,NaN
1,https://gratka.pl/nieruchomosci/mieszkania/krakow/pierwotny,https://gratka.pl/nieruchomosci/mieszkania/krakow/pierwotny?page=1,1,"Kraków, małopolskie","Nowe mieszkanie Kraków, ul. Borowego",https://gratka.pl/nieruchomosci/nowe-mieszkanie-krakow-ul-borowego/ob/23636905,NaN,58.43,3,3,NaN
2,https://gratka.pl/nieruchomosci/mieszkania/krakow/pierwotny,https://gratka.pl/nieruchomosci/mieszkania/krakow/pierwotny?page=1,1,"Kraków, małopolskie",Nowe mieszkanie Kraków,https://gratka.pl/nieruchomosci/nowe-mieszkanie-krakow/ob/25321431,NaN,72.68,3,2,NaN
3,https://gratka.pl/nieruchomosci/mieszkania/krakow/pierwotny,https://gratka.pl/nieruchomosci/mieszkania/krakow/pierwotny?page=1,1,"Kraków, małopolskie","Nowe mieszkanie Kraków, ul. Borowego",https://gratka.pl/nieruchomosci/nowe-mieszkanie-krakow-ul-borowego/ob/23636913,NaN,35.40,1,0,NaN
4,https://gratka.pl/nieruchomosci/mieszkania/krakow/pierwotny,https://gratka.pl/nieruchomosci/mieszkania/krakow/pierwotny?page=1,1,"Kraków, małopolskie","Nowe mieszkanie Kraków, ul. Facimiech",https://gratka.pl/nieruchomosci/nowe-mieszkanie-krakow-ul-facimiech/ob/25557719,NaN,61.99,3,,11082zł/m2
...,...,...,...,...,...,...,...,...,...,...,...
5368,https://gratka.pl/nieruchomosci/mieszkania/krakow/pierwotny,https://gratka.pl/nieruchomosci/mieszkania/krakow/pierwotny?page=168,168,"Kraków, Krowodrza, małopolskie","Nowe mieszkanie Kraków Krowodrza, ul. Pachońskiego",https://gratka.pl/nieruchomosci/nowe-mieszkanie-krakow-krowodrza-ul-pachonskiego/ob/21771651,NaN,27.93,1,5,NaN
5369,https://gratka.pl/nieruchomosci/mieszkania/krakow/pierwotny,https://gratka.pl/nieruchomosci/mieszkania/krakow/pierwotny?page=168,168,"Kraków, Krowodrza, małopolskie","Nowe mieszkanie Kraków Krowodrza, ul. Pachońskiego",https://gratka.pl/nieruchomosci/nowe-mieszkanie-krakow-krowodrza-ul-pachonskiego/ob/21771671,NaN,28.93,1,0,NaN
5370,https://gratka.pl/nieruchomosci/mieszkania/krakow/pierwotny,https://gratka.pl/nieruchomosci/mieszkania/krakow/pierwotny?page=168,168,"Kraków, Krowodrza, małopolskie","Nowe mieszkanie Kraków Krowodrza, ul. Pachońskiego",https://gratka.pl/nieruchomosci/nowe-mieszkanie-krakow-krowodrza-ul-pachonskiego/ob/21771689,NaN,28.92,1,2,NaN
5371,https://gratka.pl/nieruchomosci/mieszkania/krakow/pierwotny,https://gratka.pl/nieruchomosci/mieszkania/krakow/pierwotny?page=168,168,"Kraków, Krowodrza, małopolskie","Nowe mieszkanie Kraków Krowodrza, ul. Pachońskiego",https://gratka.pl/nieruchomosci/nowe-mieszkanie-krakow-krowodrza-ul-pachonskiego/ob/21771683,NaN,28.92,1,1,NaN


In [30]:
# zapisywanie pliku z danymi
gratka_df.to_csv("gratka_dane2022.csv", index=False)

## Wstępne oczyszczanie danych

In [31]:
# wczytywanie danych
gratka_df = pd.read_csv("gratka_dane2022.csv")

In [32]:
#pd.set_option('max_colwidth', 300)
gratka_df

,url_base,url_page,page,location,title,link,description,area_m2,n_rooms,n_floor,price_per_m2
0,https://gratka.pl/nieruchomosci/mieszkania/krakow/pierwotny,https://gratka.pl/nieruchomosci/mieszkania/krakow/pierwotny?page=1,1.0,"Kraków, małopolskie","Nowe mieszkanie Kraków, ul. Borowego",https://gratka.pl/nieruchomosci/nowe-mieszkanie-krakow-ul-borowego/ob/23636851,NaN,97.70,5,1.0,NaN
1,https://gratka.pl/nieruchomosci/mieszkania/krakow/pierwotny,https://gratka.pl/nieruchomosci/mieszkania/krakow/pierwotny?page=1,1.0,"Kraków, małopolskie","Nowe mieszkanie Kraków, ul. Borowego",https://gratka.pl/nieruchomosci/nowe-mieszkanie-krakow-ul-borowego/ob/23636905,NaN,58.43,3,3.0,NaN
2,https://gratka.pl/nieruchomosci/mieszkania/krakow/pierwotny,https://gratka.pl/nieruchomosci/mieszkania/krakow/pierwotny?page=1,1.0,"Kraków, małopolskie",Nowe mieszkanie Kraków,https://gratka.pl/nieruchomosci/nowe-mieszkanie-krakow/ob/25321431,NaN,72.68,3,2.0,NaN
3,https://gratka.pl/nieruchomosci/mieszkania/krakow/pierwotny,https://gratka.pl/nieruchomosci/mieszkania/krakow/pierwotny?page=1,1.0,"Kraków, małopolskie","Nowe mieszkanie Kraków, ul. Borowego",https://gratka.pl/nieruchomosci/nowe-mieszkanie-krakow-ul-borowego/ob/23636913,NaN,35.40,1,0.0,NaN
4,https://gratka.pl/nieruchomosci/mieszkania/krakow/pierwotny,https://gratka.pl/nieruchomosci/mieszkania/krakow/pierwotny?page=1,1.0,"Kraków, małopolskie","Nowe mieszkanie Kraków, ul. Facimiech",https://gratka.pl/nieruchomosci/nowe-mieszkanie-krakow-ul-facimiech/ob/25557719,NaN,61.99,3,NaN,11082zł/m2
...,...,...,...,...,...,...,...,...,...,...,...
5371,https://gratka.pl/nieruchomosci/mieszkania/krakow/pierwotny,https://gratka.pl/nieruchomosci/mieszkania/krakow/pierwotny?page=168,168.0,"Kraków, Krowodrza, małopolskie","Nowe mieszkanie Kraków Krowodrza, ul. Pachońskiego",https://gratka.pl/nieruchomosci/nowe-mieszkanie-krakow-krowodrza-ul-pachonskiego/ob/21771651,NaN,27.93,1,5.0,NaN
5372,https://gratka.pl/nieruchomosci/mieszkania/krakow/pierwotny,https://gratka.pl/nieruchomosci/mieszkania/krakow/pierwotny?page=168,168.0,"Kraków, Krowodrza, małopolskie","Nowe mieszkanie Kraków Krowodrza, ul. Pachońskiego",https://gratka.pl/nieruchomosci/nowe-mieszkanie-krakow-krowodrza-ul-pachonskiego/ob/21771671,NaN,28.93,1,0.0,NaN
5373,https://gratka.pl/nieruchomosci/mieszkania/krakow/pierwotny,https://gratka.pl/nieruchomosci/mieszkania/krakow/pierwotny?page=168,168.0,"Kraków, Krowodrza, małopolskie","Nowe mieszkanie Kraków Krowodrza, ul. Pachońskiego",https://gratka.pl/nieruchomosci/nowe-mieszkanie-krakow-krowodrza-ul-pachonskiego/ob/21771689,NaN,28.92,1,2.0,NaN
5374,https://gratka.pl/nieruchomosci/mieszkania/krakow/pierwotny,https://gratka.pl/nieruchomosci/mieszkania/krakow/pierwotny?page=168,168.0,"Kraków, Krowodrza, małopolskie","Nowe mieszkanie Kraków Krowodrza, ul. Pachońskiego",https://gratka.pl/nieruchomosci/nowe-mieszkanie-krakow-krowodrza-ul-pachonskiego/ob/21771683,NaN,28.92,1,1.0,NaN


In [33]:
#rozmiar df
gratka_df.shape

(5376, 11)

In [34]:
#podstawowe informacje
gratka_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5376 entries, 0 to 5375
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   url_base      5376 non-null   object 
 1   url_page      5375 non-null   object 
 2   page          5375 non-null   float64
 3   location      5374 non-null   object 
 4   title         5375 non-null   object 
 5   link          5373 non-null   object 
 6   description   2660 non-null   object 
 7   area_m2       5371 non-null   float64
 8   n_rooms       5360 non-null   object 
 9   n_floor       5216 non-null   float64
 10  price_per_m2  3312 non-null   object 
dtypes: float64(3), object(8)
memory usage: 462.1+ KB


In [35]:
#sprawdzenie ilości braków
gratka_df.isnull().sum()

url_base           0
url_page           1
page               1
location           2
title              1
link               3
description     2716
area_m2            5
n_rooms           16
n_floor          160
price_per_m2    2064
dtype: int64

In [36]:
#uzupełnianie braków/ usuwanie braków
gratka_df["description"].fillna("brak", inplace=True)

In [38]:
gratka_df.dropna(inplace=True)

In [39]:
gratka_df.isnull().sum()

url_base        0
url_page        0
page            0
location        0
title           0
link            0
description     0
area_m2         0
n_rooms         0
n_floor         0
price_per_m2    0
dtype: int64

In [40]:
#zmiana typów danych
gratka_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3235 entries, 13 to 5375
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   url_base      3235 non-null   object 
 1   url_page      3235 non-null   object 
 2   page          3235 non-null   float64
 3   location      3235 non-null   object 
 4   title         3235 non-null   object 
 5   link          3235 non-null   object 
 6   description   3235 non-null   object 
 7   area_m2       3235 non-null   float64
 8   n_rooms       3235 non-null   object 
 9   n_floor       3235 non-null   float64
 10  price_per_m2  3235 non-null   object 
dtypes: float64(3), object(8)
memory usage: 303.3+ KB


In [41]:
gratka_df["n_rooms"].value_counts()

2               1122
3               1119
4                558
1                343
5                 80
6                 10
7                  2
więcej niż 8       1
Name: n_rooms, dtype: int64

In [42]:
more8rooms = gratka_df.query("n_rooms == 'więcej niż 8' ")
more8rooms

,url_base,url_page,page,location,title,link,description,area_m2,n_rooms,n_floor,price_per_m2
2548,https://gratka.pl/nieruchomosci/mieszkania/krakow/pierwotny,https://gratka.pl/nieruchomosci/mieszkania/krakow/pierwotny?page=80,80.0,"Kraków, Czyżyny Stare, małopolskie",Nowe mieszkanie Kraków Czyżyny Stare,https://gratka.pl/nieruchomosci/nowe-mieszkanie-krakow-czyzyny-stare/ob/24682183,CZYŻYNY - OFERTA IDEALNA DLAINWESTORÓW. Prezentujemy Państwu ofertę mieszkań inwestycyjnych. Lokale mają powierzchnie po 230 m2 i są podzielone na 6,227.47,więcej niż 8,4.0,8750zł/m2


In [43]:
gratka_df.drop(index=[2548], inplace=True)

In [44]:
#sprawdzenie
gratka_df["n_rooms"].value_counts()

2    1122
3    1119
4     558
1     343
5      80
6      10
7       2
Name: n_rooms, dtype: int64

In [45]:
gratka_df["n_rooms"] = gratka_df["n_rooms"].astype(int)

In [46]:
gratka_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3234 entries, 13 to 5375
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   url_base      3234 non-null   object 
 1   url_page      3234 non-null   object 
 2   page          3234 non-null   float64
 3   location      3234 non-null   object 
 4   title         3234 non-null   object 
 5   link          3234 non-null   object 
 6   description   3234 non-null   object 
 7   area_m2       3234 non-null   float64
 8   n_rooms       3234 non-null   int64  
 9   n_floor       3234 non-null   float64
 10  price_per_m2  3234 non-null   object 
dtypes: float64(3), int64(1), object(7)
memory usage: 303.2+ KB


In [47]:

gratka_df["price_per_m2"] = gratka_df["price_per_m2"].str.replace("zł/m2", "").astype(int)

In [48]:
#df po przekształceniach
gratka_df.reset_index(drop=True, inplace=True)
gratka_df

,url_base,url_page,page,location,title,link,description,area_m2,n_rooms,n_floor,price_per_m2
0,https://gratka.pl/nieruchomosci/mieszkania/krakow/pierwotny,https://gratka.pl/nieruchomosci/mieszkania/krakow/pierwotny?page=1,1.0,"Kraków, Grzegórzki, małopolskie","Nowe mieszkanie Kraków Grzegórzki, ul. Mogilska",https://gratka.pl/nieruchomosci/nowe-mieszkanie-krakow-grzegorzki-ul-mogilska/ob/24563611,Sprzedam bezpośrednio nowe mieszkanie w budynku oddanym do użytku w 2021 roku. Mieszkanie w wysokim standardzie z elementami smart home. Aranżacja wg. projektu,63.18,3,4.0,16433
1,https://gratka.pl/nieruchomosci/mieszkania/krakow/pierwotny,https://gratka.pl/nieruchomosci/mieszkania/krakow/pierwotny?page=1,1.0,"Kraków, Prądnik Biały, małopolskie","Nowe mieszkanie Kraków Prądnik Biały, ul. Pachońskiego",https://gratka.pl/nieruchomosci/nowe-mieszkanie-krakow-pradnik-bialy-ul-pachonskiego/ob/23539187,"ROZKŁADMieszkanie zlokalizowane na 7 piętrze (klatka 1) o powierzchni 71,56 m2 z balkonem (5,50 m2) i tarasem (21 m2)STANDARDPoznaj nowoczesne budowni",71.56,3,7.0,11900
2,https://gratka.pl/nieruchomosci/mieszkania/krakow/pierwotny,https://gratka.pl/nieruchomosci/mieszkania/krakow/pierwotny?page=1,1.0,"Kraków, Prądnik Biały, małopolskie","Nowe mieszkanie Kraków Prądnik Biały, ul. Pachońskiego",https://gratka.pl/nieruchomosci/nowe-mieszkanie-krakow-pradnik-bialy-ul-pachonskiego/ob/23539193,"ROZKŁADMieszkanie o powierzchni 68,37 m2 zlokalizowane na parterze z dwoma ogródkami ( 47 i 15 m2) (klatka II) STANDARDPoznaj nowoczesne budownictwo",68.37,3,0.0,9900
3,https://gratka.pl/nieruchomosci/mieszkania/krakow/pierwotny,https://gratka.pl/nieruchomosci/mieszkania/krakow/pierwotny?page=1,1.0,"Kraków, Prądnik Biały, małopolskie","Nowe mieszkanie Kraków Prądnik Biały, ul. Pachońskiego",https://gratka.pl/nieruchomosci/nowe-mieszkanie-krakow-pradnik-bialy-ul-pachonskiego/ob/23538491,"ROZKŁADMieszkanie z balkonem (5,35 m2), o powierzchni 66,83 m2, zlokalizowane na II piętrze budynku (klatka 2).STANDARDPoznaj nowoczesne budownictwo m",66.83,3,2.0,10200
4,https://gratka.pl/nieruchomosci/mieszkania/krakow/pierwotny,https://gratka.pl/nieruchomosci/mieszkania/krakow/pierwotny?page=1,1.0,"Kraków, Prądnik Biały, małopolskie","Nowe mieszkanie Kraków Prądnik Biały, ul. Pachońskiego",https://gratka.pl/nieruchomosci/nowe-mieszkanie-krakow-pradnik-bialy-ul-pachonskiego/ob/23537767,"ROZKŁADMieszkanie zlokalizowane na 4 piętrze (klatka 3) o powierzchni 90,27 m2 z loggią STANDARDPoznaj nowoczesne budownictwo mieszkaniowe wielorodzin",90.27,5,4.0,9800
...,...,...,...,...,...,...,...,...,...,...,...
3229,https://gratka.pl/nieruchomosci/mieszkania/krakow/pierwotny,https://gratka.pl/nieruchomosci/mieszkania/krakow/pierwotny?page=168,168.0,"Kraków, Podgórze, małopolskie","Nowe mieszkanie Kraków Podgórze, ul. Rydlówka",https://gratka.pl/nieruchomosci/nowe-mieszkanie-krakow-podgorze-ul-rydlowka/ob/19793897,"Nova Capital ma przyjemność zaprezentować Państwu mieszkanie o powierzchni 37,33 m2 w nowo powstającej prestiżowej inwestycji przy ul. Rydlówka. Osied",37.33,2,2.0,15500
3230,https://gratka.pl/nieruchomosci/mieszkania/krakow/pierwotny,https://gratka.pl/nieruchomosci/mieszkania/krakow/pierwotny?page=168,168.0,"Kraków, Bronowice, małopolskie",Nowe mieszkanie Kraków Bronowice,https://gratka.pl/nieruchomosci/nowe-mieszkanie-krakow-bronowice/ob/19913003,Victoria Estate prezentuje 4-pokojowe mieszkanie z balkonem - Bronowice!Nieruchomość:Mieszkanie zlokalizowane na I-piętrze nowego apartamentowca w dzi,127.99,4,1.0,9990
3231,https://gratka.pl/nieruchomosci/mieszkania/krakow/pierwotny,https://gratka.pl/nieruchomosci/mieszkania/krakow/pierwotny?page=168,168.0,"Kraków, Mistrzejowice, małopolskie",Nowe mieszkanie Kraków Mistrzejowice,https://gratka.pl/nieruchomosci/nowe-mieszkanie-krakow-mistrzejowice/ob/19912989,Victoria Estate prezentuje 3-pokojowe mieszkanie z ogrodem na wschodzie Krakowa! Nieruchomość:Mieszkanie zlokalizowane na I-piętrze X-piętrowej inwest,67.53,3,1.0,8300
3232

In [49]:
#rozmiar df po usunięciu braków
gratka_df.shape

(3234, 11)

In [50]:
#podstawowe informacje po przekształceniach
gratka_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3234 entries, 0 to 3233
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   url_base      3234 non-null   object 
 1   url_page      3234 non-null   object 
 2   page          3234 non-null   float64
 3   location      3234 non-null   object 
 4   title         3234 non-null   object 
 5   link          3234 non-null   object 
 6   description   3234 non-null   object 
 7   area_m2       3234 non-null   float64
 8   n_rooms       3234 non-null   int64  
 9   n_floor       3234 non-null   float64
 10  price_per_m2  3234 non-null   int64  
dtypes: float64(3), int64(2), object(6)
memory usage: 278.0+ KB


In [51]:
#wstępne statystyki danych
gratka_df.describe()

,page,area_m2,n_rooms,n_floor,price_per_m2
count,3234.000000,3234.000000,3234.000000,3234.000000,3234.000000
mean,77.850649,58.110622,2.674706,2.439085,12006.822202
std,47.762671,23.756152,0.987361,2.270352,2846.554730
min,1.000000,13.300000,1.000000,0.000000,4900.000000
25%,35.000000,42.610000,2.000000,1.000000,10056.250000
50%,71.000000,54.265000,3.000000,2.000000,11411.500000
75%,119.000000,67.870000,3.000000,4.000000,13500.000000
max,168.000000,334.200000,7.000000,14.000000,37702.000000


In [52]:
# zapisywanie pliku z danymi
gratka_df.to_csv("gratka_dane_new2022.csv", index=False)